In [263]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
model_path = '/Users/lambert/Documents/Python_code/tensorflow_code/Titanic/Titanic_model/'

In [264]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [265]:
# 1. Data clearning
# Age - find the median and fill to 
# may be try df_test_Age with its own median
# fill_df_test_Age = df_test.Age.median()
fill_df_Age = df_train.Age.median()
df_train.Age.fillna(fill_df_Age, inplace=True)
df_test.Age.fillna(fill_df_Age, inplace=True)

# Cabin - fill nan with 'fill' and select first cabin
df_train.Cabin.fillna('fill',inplace=True)
df_train.Cabin = df_train.Cabin.map(lambda x:x.split(' ')[0])
df_test.Cabin.fillna('fill', inplace=True)
df_test.Cabin = df_test.Cabin.map(lambda x:x.split(' ')[0])

# Sex - male: 1, female: 1
df_train.Sex.replace({'male':1, 'female':0}, inplace=True)
df_test.Sex.replace({'male':1, 'female':0}, inplace=True)

df_train = df_train.dropna(axis=0, how='any')
df_test.Fare.fillna(df_test.Fare.mean(), inplace=True)

# Create a 'Deceased' column for sconed class
df_train['Deceased'] = df_train.Survived.apply(lambda s: int(not s))

In [266]:
df_train.iloc[606]

PassengerId                            608
Survived                                 1
Pclass                                   1
Name           Daniel, Mr. Robert Williams
Sex                                      1
Age                                     27
SibSp                                    0
Parch                                    0
Ticket                              113804
Fare                                  30.5
Cabin                                 fill
Embarked                                 S
Deceased                                 0
Name: 607, dtype: object

In [267]:
# 2. Select the features
sFeatures = ['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',             
 'Cabin',             
 'Embarked']

X = df_train[sFeatures]
y = df_train[['Survived','Deceased']]
y = y.values.reshape(len(y),2)
X_test = df_test[sFeatures]

In [268]:
# 3. Label Encoder
# X_test has elements that X doesn't have
# create a Cabin_labels cover X and X_test
X_Cabin_unique = X['Cabin'].unique()
X_lack = X_test.Cabin[X_test['Cabin'].isin(X_Cabin_unique)==0].values
Cabin_labels = np.append(X_Cabin_unique, X_lack)
# print(X_Cabin_unique.shape, X_Cabin_unique)
# print(Cabin_labels.shape, Cabin_labels)

# LabelEncoder categorical Cabin and Embarked
# le_Cabin = LabelEncoder().fit(np.array(X['Cabin'].tolist()))
le_Cabin = LabelEncoder().fit(Cabin_labels)
le_Embarked = LabelEncoder().fit(np.array(X['Embarked'].tolist()))

X['Cabin'] = le_Cabin.transform(X['Cabin'])
X['Embarked'] = le_Embarked.transform(X['Embarked'])
X_test['Cabin'] = le_Cabin.transform(X_test['Cabin'])
X_test['Embarked'] = le_Embarked.transform(X_test['Embarked'])
X = X.astype(np.float32)
y = y.astype(np.float32)

/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [269]:
# Validation
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [270]:
X_train[0:10]

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
417,2.0,0.0,18.0,0.0,2.0,13.000000,180.0,2.0
63,3.0,1.0,4.0,3.0,2.0,27.900000,180.0,2.0
589,3.0,1.0,28.0,0.0,0.0,8.050000,180.0,2.0
483,3.0,0.0,63.0,0.0,0.0,9.587500,180.0,2.0
206,3.0,1.0,32.0,1.0,0.0,15.850000,180.0,2.0
228,2.0,1.0,18.0,0.0,0.0,13.000000,180.0,2.0
871,1.0,0.0,47.0,1.0,1.0,52.554199,130.0,2.0
510,3.0,1.0,29.0,0.0,0.0,7.750000,180.0,1.0
241,3.0,0.0,28.0,1.0,0.0,15.500000,180.0,1.0
865,2.0,0.0,42.0,0.0,0.0,13.000000,180.0,2.0


In [271]:
y_train[0:10]

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

### Tensor Graph

In [272]:
def next_batch(batch, X, y):
    num = np.arange(len(X))
    np.random.shuffle(num)
    select = num[0:batch]
    x_batch = X.iloc[select]
    y_batch = y[select]
    
    return(x_batch, y_batch)

In [273]:
batch_size = 200

def optimize(num_iterations):
    total_loss = 0
    for i in range(num_iterations):
        x_batch, y_batch = next_batch(batch_size, X_train_data, y_train_data)
        feed_dict_train = {X_place: x_batch, y_place: y_batch}
        _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
        total_loss += loss
        print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
    print ('Training complete!')

In [274]:
# Tensorflow variables
features= len(sFeatures)
X_place = tf.placeholder(tf.float32, shape=[None,features])
y_place = tf.placeholder(tf.float32, shape=[None,2])
W = tf.Variable(tf.random_normal([features, 2]), name='weights')
b = tf.Variable(tf.zeros(2), name='bias')
logits = tf.matmul(X_place, W) + b
y_pred = tf.nn.softmax(logits)

In [275]:
# Cross Entropy
learning_rate = 0.001
cross_entropy = -tf.reduce_sum(y_place * tf.log(y_pred + 1e-10), axis=1)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_train)
cost = tf.reduce_mean(cross_entropy)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
saver = tf.train.Saver()

In [276]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [277]:
# # training starting option 1
# for epoch in range(10000):
#     total_loss = 0.
#     x_batch, y_batch = next_batch(batch_size, X_train, y_train)
#     feed_dict_train = {X_place: x_batch, y_place: y_batch}
#     _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
#     total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [278]:
# sess.close()

In [282]:
# training starting option 1
for epoch in range(100):
    total_loss = 0.
    for i in range(len(X_train)):
        feed = {X_place: X_train, y_place: y_train}
        _, loss = sess.run([train_op, cost], feed_dict=feed)
        total_loss += loss
    print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
#     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
#     print('predict: {:.6f}'.format(y_pred))
print ('Training complete!')

feed_dict_train = {X_place: X_train}
prediction = sess.run(y_pred, feed_dict=feed_dict_train)
correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train_data,axis=1))
# evaluate the accuracy
accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

Epoch: 0001, total loss=5541.680708885
Epoch: 0002, total loss=5541.540865898
Epoch: 0003, total loss=5541.413506508
Epoch: 0004, total loss=5541.293797493
Epoch: 0005, total loss=5541.177625656
Epoch: 0006, total loss=5541.061377525
Epoch: 0007, total loss=5540.940877914
Epoch: 0008, total loss=5540.811052322
Epoch: 0009, total loss=5540.664940834
Epoch: 0010, total loss=5540.492752075
Epoch: 0011, total loss=5540.281257629
Epoch: 0012, total loss=5540.016325951
Epoch: 0013, total loss=5539.691099167
Epoch: 0014, total loss=5539.312497139
Epoch: 0015, total loss=5538.894763947
Epoch: 0016, total loss=5538.449623108
Epoch: 0017, total loss=5537.984559059
Epoch: 0018, total loss=5537.504734993
Epoch: 0019, total loss=5537.014330864
Epoch: 0020, total loss=5536.516591072
Epoch: 0021, total loss=5536.014182091
Epoch: 0022, total loss=5535.508934975
Epoch: 0023, total loss=5535.002261162
Epoch: 0024, total loss=5534.494976997
Epoch: 0025, total loss=5533.987763405
Epoch: 0026, total loss=5

In [206]:
save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
# sess.close()

In [207]:
# evaluate the training data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_train = {X_place: X_train_data}
    prediction = sess.run(y_pred, feed_dict=feed_dict_train)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train_data,axis=1))
# evaluate the accuracy
accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /Users/lambert/Documents/Python_code/tensorflow_code/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 63.06%


In [210]:
# evaluate the validate data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_valid = {X_place: X_val_data}
    prediction = sess.run(y_pred, feed_dict=feed_dict_valid)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_val_data,axis=1))
# evaluate the accuracy

accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /Users/lambert/Documents/Python_code/tensorflow_code/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 64.13%


In [224]:
# prediction for test data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_test = {X_place: X_test}
    prediction = sess.run(y_pred, feed_dict=feed_dict_test)
    y_predict = np.argmax(prediction, axis=1)
# evaluate the accuracy

INFO:tensorflow:Restoring parameters from /Users/lambert/Documents/Python_code/tensorflow_code/Titanic/Titanic_model/Titanic_model.ckpt


In [223]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_random.csv')

407

In [226]:
y_predict.shape

(418,)

In [220]:
# print_accuracy()
# plot_example_errors()
# optimize(num_iterations=10)
print_accuracy()
# plot_example_errors()
# plot_weights()

NameError: name 'print_accuracy' is not defined

In [ ]:
File_Writer=tf.summary.FileWriter(''/Users/lambert/Documents/Python_code/tensorflow_code/Titanic'
                                  ,sess.graph)

In [ ]:
device_lib.list_local_devices()

Logistic Regression

In [ ]:
# Logistic Regression
lr = LogisticRegression(C=1, max_iter=100, solver='lbfgs').fit(X_train, y_train)
yp = lr.predict(X_train)
y_predict = lr.predict(X_test)

passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_knn.csv')
# passengerid = np.array(df_train.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': yp}).set_index('PassengerId')
# submission.to_csv('y_train_predict.csv')

Gradient Boosting Classifier

In [ ]:
# Gradient Boosting Classifier
# gbdt = GradientBoostingClassifier(n_estimators=300, max_depth=5).fit(X_train, y_train)
# y_predict = gbdt.predict(X_test)

# passengerid = np.array(df_test.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
# submission.to_csv('submission.csv')

Random forests

In [ ]:
# RandomForest
rfc = RandomForestClassifier(n_estimators=200, max_depth=3).fit(X_train, y_train)
y_predict = rfc.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_random.csv')

Stochastic Gradient Descent (SDG)

In [ ]:
# Stochastic Gradient Descent (SGD) easily scale with more than 10^5 samples and features
sgd = SGDClassifier(alpha=0.01, max_iter=1000).fit(X_train, y_train)
y_predict = sgd.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_sdg.csv')